In [1]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import pickle
import matplotlib.pyplot as plt
import timm

In [2]:
def accuracy(output, target):
    with torch.no_grad():
        pred = torch.argmax(output, dim=1)
        assert pred.shape[0] == len(target)
        correct = 0
        correct += torch.sum(pred == target).item()
    return correct / len(target)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
with open('images_path.pickle','rb') as f:
    images_path = pickle.load(f)

In [5]:
batch_size = 64
MODEL_PATH ="saved"
folder_path = '/opt/ml/input/data/train/images/'
LEARNING_RATE = 0.001
EPOCHS = 20
val_rate = 0.1

In [6]:
class AgeClassifier(nn.Module):
    def __init__(self, num_of_classes = 3):
        super().__init__()
        self.m = timm.create_model('efficientnet_b1',pretrained=True)
        self.fc = nn.Linear(self.m.classifier.out_features, num_of_classes)
    
    def forward(self, x):
        x = self.m(x)
        return self.fc(x)

In [7]:
class MaskAgeDataSet(Dataset):
    def __init__(self,f_path,images_path,transform=None,train=True):
        self.f_path = f_path
        self.images_path = images_path
        self.transform = transform
        self.info = pd.read_csv('image_info.csv')
        self.y = self.info['age']
        
    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        y = torch.tensor(self.y[index])

        x = Image.open(self.f_path + self.images_path[index])
        if self.transform:
            x = self.transform(x)
        
        return x, y

In [8]:
age_clf = AgeClassifier()

# for parm in age_clf.parameters():
#     parm.requires_grad = False
# for parm in age_clf.fc.parameters():
#     parm.requires_grad = True
mask_age_data = MaskAgeDataSet(f_path=folder_path,
                                        images_path=images_path,
                                        transform=transforms.Compose([transforms.RandomCrop(300),
                                                                      transforms.RandomPerspective(),
                                                                      transforms.RandomPerspective(),
                                                                      transforms.RandomHorizontalFlip(),
                                                                      transforms.RandomGrayscale(),
                                                                      Resize((260,260)),
                                                                      ToTensor(),
                                                                      Normalize(mean=(0.5, 0.5, 0.5), std=(0.2, 0.2, 0.2)),
                                                                      ]))

mask_age_data_loader = DataLoader(dataset=mask_age_data,batch_size=batch_size,
                                        shuffle=True,num_workers=4)

In [9]:
mask_age_data, validation_data = torch.utils.data.random_split(mask_age_data,[int(len(mask_age_data)*(1-val_rate)),
                                                                                    int(len(mask_age_data)*val_rate)])
mask_age_data_loader = DataLoader(dataset=mask_age_data,batch_size=batch_size,
                                        shuffle=True,drop_last=True,num_workers=4)
validation_loader = DataLoader(dataset = validation_data, batch_size = batch_size,drop_last=True,shuffle=True,num_workers=4)

In [10]:
age_clf = age_clf.to(device)
criterion = nn.CrossEntropyLoss(weight = torch.tensor([1.,1.,6.]).to(device)) # 수정필요
optimizer = optim.Adam(age_clf.parameters(), lr=LEARNING_RATE)



for e in range(1, EPOCHS+1):
    epoch_loss = 0
    epoch_acc = 0
    for X_batch, y_batch in mask_age_data_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()        
        y_pred = age_clf(X_batch)
        
        loss = criterion(y_pred, y_batch)
        acc = accuracy(y_pred, y_batch)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item() 
        epoch_acc += acc
    
    age_clf.eval()
    vaild_acc = 0
    for data, labels, in validation_loader:
        data, labels = data.to(device), labels.to(device)
        
        y_pred = age_clf(data)
        vacc = accuracy(y_pred, labels)
        vaild_acc += vacc
    age_clf.train()
    va = f'{vaild_acc/len(validation_loader):.3f}' 
    l = f'{epoch_loss/len(mask_age_data_loader):.5f}'
    a = f'{epoch_acc/len(mask_age_data_loader):.3f}'
    print(f'Epoch {e+0:03}: | Loss: {l} | Acc: {a} | VAcc: {va}')
    if not os.path.exists(MODEL_PATH):
        os.makedirs(MODEL_PATH)
    torch.save(age_clf.state_dict(), os.path.join(MODEL_PATH, f'age_model_{e+0:03}.pt'))

Epoch 001: | Loss: 0.45475 | Acc: 0.827 | VAcc: 0.839
Epoch 002: | Loss: 0.29696 | Acc: 0.884 | VAcc: 0.904
Epoch 003: | Loss: 0.25403 | Acc: 0.900 | VAcc: 0.893
Epoch 004: | Loss: 0.22602 | Acc: 0.910 | VAcc: 0.931
Epoch 005: | Loss: 0.20469 | Acc: 0.923 | VAcc: 0.906
Epoch 006: | Loss: 0.16847 | Acc: 0.936 | VAcc: 0.946
Epoch 007: | Loss: 0.17149 | Acc: 0.934 | VAcc: 0.940
Epoch 008: | Loss: 0.13490 | Acc: 0.952 | VAcc: 0.849
Epoch 009: | Loss: 0.13970 | Acc: 0.950 | VAcc: 0.937
Epoch 010: | Loss: 0.12453 | Acc: 0.955 | VAcc: 0.958
Epoch 011: | Loss: 0.11067 | Acc: 0.961 | VAcc: 0.943
Epoch 012: | Loss: 0.11655 | Acc: 0.957 | VAcc: 0.950
Epoch 013: | Loss: 0.10195 | Acc: 0.964 | VAcc: 0.935
Epoch 014: | Loss: 0.11191 | Acc: 0.958 | VAcc: 0.968
Epoch 015: | Loss: 0.09013 | Acc: 0.968 | VAcc: 0.937
Epoch 016: | Loss: 0.10404 | Acc: 0.964 | VAcc: 0.949
Epoch 017: | Loss: 0.08509 | Acc: 0.968 | VAcc: 0.971
Epoch 018: | Loss: 0.08506 | Acc: 0.970 | VAcc: 0.977
Epoch 019: | Loss: 0.07902 |

In [ ]:
# a = nn.CrossEntropyLoss(weight=torch.tensor([1.,2.,2]))
# b = torch.tensor([[1.,0.,0.]],requires_grad=True)
# c = torch.LongTensor([0])
# d = torch.LongTensor([1])
# e = torch.LongTensor([2])
# print(a(b,c))
# print(a(b,d))
# print(a(b,e))

In [ ]:
'''
for i, left in enumerate(dataloader):
    print(i)
    with torch.no_grad():
        temp = model(left).view(-1, 1, 300, 300)
    right.append(temp.to('cpu'))
    del temp
    torch.cuda.empty_cache()
'''